#### Extract cell positions from Imaris spot detection and cell tracking output

- Author: Zachary Neronha
- Last Modified: Nov 21 2018 (Dhananjay Bhaskar)

In [1]:
import numpy as np
import pandas as pd
import glob
import os

In [6]:
readpath = glob.glob("Wk2_IMS_Crop_Tracks/*/*Position.csv")

In [7]:
print(readpath)

['Wk2_IMS_Crop_Tracks/3d sc xmas 9-20-18 week 2_B10_spot11_Statistics/3d sc xmas 9-20-18 week 2_B10_spot11_Position.csv', 'Wk2_IMS_Crop_Tracks/3d sc xmas 9-20-18 week 2_A12_spots15_Statistics/3d sc xmas 9-20-18 week 2_A12_spots15_Position.csv', 'Wk2_IMS_Crop_Tracks/3d sc xmas 9-20-18 week 2_A07_spots17_Statistics/3d sc xmas 9-20-18 week 2_A07_spots17_Position.csv', 'Wk2_IMS_Crop_Tracks/3d sc xmas 9-20-18 week 2_A08_spots2_Statistics/3d sc xmas 9-20-18 week 2_A08_spots2_Position.csv']


In [14]:
for path in readpath:
    
    filename = os.path.basename(path)
    wellname = filename.split("_")[1]
    
    df = pd.read_csv(path,skiprows=[0,1,2])
    inputdata = np.array(df)

    #split up data into constitutant columns 
    Xdata = inputdata[:,0]
    Ydata = inputdata[:,1]
    Zdata = inputdata[:,2]
    Tdata = inputdata[:,6]
    TrackID = inputdata[:,7]

    #rescale data as needed
    TrackID = TrackID-min(TrackID)
    #Xdata = Xdata-min(Xdata)
    #Ydata = Ydata-min(Ydata)
    #Zdata = Zdata-min(Zdata)
    Tdata = Tdata - 1

    #how many actual cells
    #TrackIDl = np.isnan(np.float64(TrackID))
    #tiReal = TrackID[TrackIDl==False]
    #numCELL = len(np.unique(tiReal))
    numCELL = np.max(TrackID)+1
    
    #preallocate memory for final dataframes
    xStore = np.empty((numCELL,len(np.unique(Tdata))))
    yStore = np.empty((numCELL,len(np.unique(Tdata))))
    zStore = np.empty((numCELL,len(np.unique(Tdata))))
    xStore[:] = np.nan
    yStore[:] = np.nan
    zStore[:] = np.nan
    
    #loop through time frames
    for time in range(0,len(np.unique(Tdata))):
    #for time in range(0,10):
        #pull out appropriate temporal data
        tempIX = Tdata == time
        tempT = Tdata[tempIX==True]
        tempX = Xdata[tempIX==True]
        tempY = Ydata[tempIX==True]
        tempZ = Zdata[tempIX==True]
        tempID = TrackID[tempIX==True]

        for ii in range(0,len(np.unique(tempID))):
            #determine which cell we are indexing
            cell = tempID[ii]
            #if the cell is registered
            if np.isnan(np.float64(cell))==False:
                cell = np.int64(cell)
                xStore[cell,time] = tempX[np.int64(ii)]
                yStore[cell,time] = tempY[np.int64(ii)]
                zStore[cell,time] = tempZ[np.int64(ii)]

    
    dx = pd.DataFrame(xStore)
    dy = pd.DataFrame(yStore)
    dz = pd.DataFrame(zStore)
    
    dirname = os.path.dirname(path)
    outfilename = os.sep.join([dirname, wellname + "_combineddata.xls"])
    
    writer = pd.ExcelWriter(outfilename, engine='xlsxwriter')
    dx.to_excel(writer, sheet_name='xStore')
    dy.to_excel(writer, sheet_name='yStore')
    dz.to_excel(writer, sheet_name='zStore')
    
    print(wellname)
    print("Number of tracks: " + repr(numCELL))

B10
Number of tracks: 27
A12
Number of tracks: 25
A07
Number of tracks: 30
A08
Number of tracks: 24
